# Limpieza de Datos del Dataset IGAC

Este notebook procesa el dataset completo aplicando funciones de limpieza.

## Objetivos
1. Procesar el dataset completo en chunks
2. Limpiar nombres de municipios y departamentos
3. Validar y corregir fechas
4. Limpiar valores numéricos
5. Manejar valores nulos
6. Eliminar duplicados
7. Guardar datos limpios en formato Parquet

In [13]:
# Importar librerías
import sys
from pathlib import Path
import polars as pl

# Añadir el path del proyecto para importar módulos propios
sys.path.append('../src')

from etl.data_loader import load_full_dataset_lazy, save_to_parquet
from etl.data_cleaner import apply_all_cleaning
from utils.config import CLEANED_PARQUET_FILE

# Configuración visual de Polars (opcional, ayuda a ver más datos)
pl.Config.set_tbl_cols(20)
pl.Config.set_fmt_str_lengths(50)

print("✅ Librerías importadas y entorno configurado.")

✅ Librerías importadas y entorno configurado.


## 1. Carga Lazy y Exploración Inicial

In [15]:
# Cargar dataset en modo lazy (Metadatos solamente)
lf = load_full_dataset_lazy()

print(f"📡 Conexión establecida (Lazy).")
print(f"📋 Columnas detectadas ({len(lf.columns)}):")
print(lf.columns)

Cargando dataset en modo lazy (sin cargar en memoria)...
✓ Dataset cargado en modo lazy
📡 Conexión establecida (Lazy).
📋 Columnas detectadas (26):
['PK', 'MATRICULA', 'FECHA_RADICA_TEXTO', 'FECHA_APERTURA_TEXTO', 'YEAR_RADICA', 'ORIP', 'DIVIPOLA', 'DEPARTAMENTO', 'MUNICIPIO', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'NUM_ANOTACION', 'ESTADO_FOLIO', 'FOLIOS_DERIVADOS', 'Dinámica_Inmobiliaria', 'COD_NATUJUR', 'NOMBRE_NATUJUR', 'NUMERO_CATASTRAL', 'NUMERO_CATASTRAL_ANTIGUO', 'DOCUMENTO_JUSTIFICATIVO', 'COUNT_A', 'COUNT_DE', 'PREDIOS_NUEVOS', 'TIENE_VALOR', 'TIENE_MAS_DE_UN_VALOR', 'VALOR']


C:\Users\carlo\AppData\Local\Temp\ipykernel_5824\3036315462.py:5: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print(f"📋 Columnas detectadas ({len(lf.columns)}):")
C:\Users\carlo\AppData\Local\Temp\ipykernel_5824\3036315462.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print(lf.columns)


## 2. Ejecución del ETL (Carga + Limpieza + Optimización)

Aplicamos todas las funciones de limpieza definidas en el módulo `data_cleaner`.

In [16]:
print("\n🚀 INICIANDO PROCESO ETL (EXTRACT, TRANSFORM, LOAD)...")
print("=" * 60)

# 1. Recolectar (EXTRACT)
# Usamos 'engine="streaming"' para evitar picos de RAM con 30M de filas
print("1️⃣  Cargando datos en memoria (Streaming activo)...")
df = lf.collect(engine="streaming")

# Guardar métricas iniciales
filas_originales = df.height
cols_originales = df.width
print(f"   ➤ Datos crudos cargados: {filas_originales:,} filas | {cols_originales} columnas")

# 2. Limpiar (TRANSFORM)
print("\n2️⃣  Aplicando reglas de limpieza y negocio...")
# Sobreescribimos 'df' para liberar la memoria de los datos sucios inmediatamente
df = apply_all_cleaning(df)

# 3. Post-Procesamiento (Ajuste final recomendado)
# Convertir cadenas vacías "" o espacios " " a Nulos reales (null)
print("   ➤ Estandarizando celdas vacías a Nulos reales...")
df = df.with_columns([
    pl.when(pl.col(pl.String).str.strip_chars().str.len_chars() == 0)
    .then(None)
    .otherwise(pl.col(pl.String))
    .name.keep()
])

# Recalcular métricas
filas_finales = df.height
cols_finales = df.width

print("\n✅ TRANSFORMACIÓN COMPLETADA.")


🚀 INICIANDO PROCESO ETL (EXTRACT, TRANSFORM, LOAD)...
1️⃣  Cargando datos en memoria (Streaming activo)...
   ➤ Datos crudos cargados: 30,903,248 filas | 26 columnas

2️⃣  Aplicando reglas de limpieza y negocio...

🚀 INICIANDO PROCESO COMPLETO DE LIMPIEZA
🛡️ SANEANDO COMILLAS DOBLES...
✓ Comillas eliminadas en estructura y datos
⚖️ Aplicando reglas de negocio específicas...
   - Eliminando: ['FECHA_APERTURA_TEXTO', 'NUMERO_CATASTRAL_ANTIGUO']
✓ Reglas de negocio aplicadas
🧹 LIMPIANDO COLUMNAS STRING...
   ➤ Columnas detectadas: ['MATRICULA', 'ORIP', 'DIVIPOLA', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'ESTADO_FOLIO', 'NOMBRE_NATUJUR']
✓ Columnas string limpiadas
→ Normalizando nombres de municipios...
Normalizando nombres de departamentos...
✓ Departamentos normalizados
Parseando columnas de fecha...
✓ Fechas parseadas
Limpiando valores numéricos...
✓ Valores numéricos limpios
Manejando valores nulos: estrategia report
Eliminando duplicados...
✓ Eliminados 0 duplicados
✅ LIMPIEZA CO

## 3. Reporte de Calidad y Resultados

In [17]:
# Reporte de Comparación
print("\n" + "=" * 60)
print("📊 REPORTE DE RESULTADOS ETL")
print("=" * 60)
print(f"Filas Originales  : {filas_originales:,}")
print(f"Filas Finales     : {filas_finales:,}")
print(f"Filas Eliminadas  : {filas_originales - filas_finales:,}")
print("-" * 30)
print(f"Columnas (Antes)  : {cols_originales}")
print(f"Columnas (Ahora)  : {cols_finales}")
print(f"Columnas Borradas : {cols_originales - cols_finales}")
print("-" * 30)
print(f"Retención de Datos: {(filas_finales / filas_originales) * 100:.2f}%")
print("=" * 60)

print("\n🔎 Vista previa de datos limpios (Primeras 5 filas):")
print(df.head(5))


📊 REPORTE DE RESULTADOS ETL
Filas Originales  : 30,903,248
Filas Finales     : 30,903,248
Filas Eliminadas  : 0
------------------------------
Columnas (Antes)  : 26
Columnas (Ahora)  : 24
Columnas Borradas : 2
------------------------------
Retención de Datos: 100.00%

🔎 Vista previa de datos limpios (Primeras 5 filas):
shape: (5, 24)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ PK  ┆ MAT ┆ FEC ┆ YEA ┆ ORI ┆ DIV ┆ DEP ┆ MUN ┆ TIP ┆ CAT ┆ … ┆ COD ┆ NOM ┆ NUM ┆ DOC ┆ COU ┆ COU ┆ PRE ┆ TIE ┆ TIE ┆ VAL │
│ --- ┆ RIC ┆ HA_ ┆ R_R ┆ P   ┆ IPO ┆ ART ┆ ICI ┆ O_P ┆ EGO ┆   ┆ _NA ┆ BRE ┆ ERO ┆ UME ┆ NT_ ┆ NT_ ┆ DIO ┆ NE_ ┆ NE_ ┆ OR  │
│ str ┆ ULA ┆ RAD ┆ ADI ┆ --- ┆ LA  ┆ AME ┆ PIO ┆ RED ┆ RIA ┆   ┆ TUJ ┆ _NA ┆ _CA ┆ NTO ┆ A   ┆ DE  ┆ S_N ┆ VAL ┆ MAS ┆ --- │
│     ┆ --- ┆ ICA ┆ CA  ┆ str ┆ --- ┆ NTO ┆ --- ┆ IO_ ┆ _RU ┆   ┆ UR  ┆ TUJ ┆ TAS ┆ _JU ┆ --- ┆ --- ┆ UEV ┆ OR  ┆ _DE ┆ f64 │
│     ┆ str ┆ _TE ┆ --- ┆     ┆

## 4. Guardado (Load)

In [18]:
# 6. Guardar en Parquet (LOAD)
print(f"\n💾 Guardando archivo procesado en: {CLEANED_PARQUET_FILE}")

# Usamos compresión 'zstd' que suele ser mejor que 'snappy' para archivos grandes,
# pero si tu config pide 'snappy', puedes dejarlo.
save_to_parquet(df, CLEANED_PARQUET_FILE, compression='zstd')

print(f"✅ Archivo guardado exitosamente.")

# Opcional: Liberar memoria si no vas a hacer más análisis en este notebook
# del df
# import gc
# gc.collect()


💾 Guardando archivo procesado en: c:\Users\carlo\Documents\KrozFu\repos\challenge-opendata\igac-anomalias-inmobiliarias\notebooks\..\data\processed\igac_cleaned.parquet
Guardando datos en formato Parquet: c:\Users\carlo\Documents\KrozFu\repos\challenge-opendata\igac-anomalias-inmobiliarias\notebooks\..\data\processed\igac_cleaned.parquet
✓ Archivo guardado: 1211.18 MB
✅ Archivo guardado exitosamente.


## 5. Estadísticas Finales (Analytics)

In [19]:
print("\n📈 ESTADÍSTICAS DEL DATASET LIMPIO")
print("=" * 40)

# Estadísticas generales (numéricas)
print("\nResumen Estadístico:")
print(df.describe())

# Validación específica: Distribución por Año
# Verificamos que la columna exista antes de agrupar
if 'YEAR_RADICA' in df.columns:
    print("\nDistribución por Año de Radicación:")
    year_dist = (
        df.group_by('YEAR_RADICA')
        .agg(pl.count().alias('Cantidad'))
        .sort('YEAR_RADICA')
    )
    print(year_dist)
else:
    print("\n⚠ La columna 'YEAR_RADICA' no se encuentra en el dataset final.")

# Validación de Nulos Finales
print("\nConteo de Nulos por Columna:")
print(df.null_count())


📈 ESTADÍSTICAS DEL DATASET LIMPIO

Resumen Estadístico:
shape: (9, 25)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ sta ┆ PK  ┆ MAT ┆ FEC ┆ YEA ┆ ORI ┆ DIV ┆ DEP ┆ MUN ┆ TIP ┆ … ┆ COD ┆ NOM ┆ NUM ┆ DOC ┆ COU ┆ COU ┆ PRE ┆ TIE ┆ TIE ┆ VAL │
│ tis ┆ --- ┆ RIC ┆ HA_ ┆ R_R ┆ P   ┆ IPO ┆ ART ┆ ICI ┆ O_P ┆   ┆ _NA ┆ BRE ┆ ERO ┆ UME ┆ NT_ ┆ NT_ ┆ DIO ┆ NE_ ┆ NE_ ┆ OR  │
│ tic ┆ str ┆ ULA ┆ RAD ┆ ADI ┆ --- ┆ LA  ┆ AME ┆ PIO ┆ RED ┆   ┆ TUJ ┆ _NA ┆ _CA ┆ NTO ┆ A   ┆ DE  ┆ S_N ┆ VAL ┆ MAS ┆ --- │
│ --- ┆     ┆ --- ┆ ICA ┆ CA  ┆ str ┆ --- ┆ NTO ┆ --- ┆ IO_ ┆   ┆ UR  ┆ TUJ ┆ TAS ┆ _JU ┆ --- ┆ --- ┆ UEV ┆ OR  ┆ _DE ┆ f64 │
│ str ┆     ┆ str ┆ _TE ┆ --- ┆     ┆ str ┆ --- ┆ str ┆ ZON ┆   ┆ --- ┆ UR  ┆ TRA ┆ STI ┆ f64 ┆ f64 ┆ OS  ┆ --- ┆ _UN ┆     │
│     ┆     ┆     ┆ XTO ┆ f64 ┆     ┆     ┆ str ┆     ┆ A   ┆   ┆ f64 ┆ --- ┆ L   ┆ FIC ┆     ┆     ┆ --- ┆ f64 ┆ _VA ┆     │
│     ┆     ┆     ┆ --- ┆     ┆     ┆     ┆   

C:\Users\carlo\AppData\Local\Temp\ipykernel_5824\2993445702.py:14: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias('Cantidad'))
